In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/study/16715robotsim/project/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Test
using Test
using Plots

## System properties

In [3]:
# simulation step size
Δt = 0.05
const g = 9.81

m = 1
μ = 0.5

w = 1.0 # width of the box
h = 0.8 # height of the box
M = [m 0 0; 0 m 0; 0 0 m*((h/2)^2+(w/2)^2)/3]

# set of contact modes
modes = [0 0 0 0; # both free
        1 0 0 0; # sticking
        1 0 -1 0; # left-slide
        1 0 1 0; # right-slide
        0 1 0 0;
        0 1 0 -1;
        0 1 0 1;
        1 1 0 0;
        1 1 -1 -1;
        1 1 1 1]

n_contacts = 2


2

## Simulation functions

In [4]:
function R_2D(θ)
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return R
end

R_2D (generic function with 1 method)

In [5]:
# constraints/contacts
function compute_a(q)
    y = q[2]
    θ = q[3]
    a1 = y - 0.5*h*cos(θ) - 0.5*w*sin(θ)
    a2 = y - 0.5*h*cos(θ) + 0.5*w*sin(θ)
    return [a1; a2]
end

function compute_a_t(q)
    p1 = R_2D(q[3])*[-w/2; -h/2] 
    p2 = R_2D(q[3])*[w/2; -h/2] 
    at1 = p1[1] + q[1]
    at2 = p2[1] + q[1]
    return [at1; at2]
end

# constraints jacobian
function compute_A(q)
    θ = q[3]
    A = [0 1 0.5*h*sin(θ)-0.5*w*cos(θ); 0 1 0.5*h*sin(θ)+0.5*w*cos(θ)]
    return A
end

function compute_dA(q, dq)
    dA = reshape(ForwardDiff.jacobian(compute_A, q)*dq, n_contacts, 3)
    return dA
end

function compute_A_tangent(q)
    A_t = ForwardDiff.jacobian(compute_a_t,q)
    return A_t
end

function compute_dA_tangent(q, dq)
    dA_t = reshape(ForwardDiff.jacobian(compute_A_tangent, q)*dq, n_contacts, 3)
    return dA_t
end

compute_dA_tangent (generic function with 1 method)

In [6]:
function contact_mode_constraints(x, contactMode)
    
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    A_all = zeros(0,3)
    A_all_f = zeros(0,3)
    dA_all = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            A_all_f = [A_all_f; A[k,:]'; A_t[k,:]']
            dA_all = [dA_all; dA[k,:]'; dA_t[k,:]']
            A_all = [A_all; A[k,:]'; A_t[k,:]']
        else
            A_all_f = [A_all_f; A[k,:]'-ss*μ*A_t[k,:]']
            dA_all = [dA_all; dA[k,:]']
            A_all = [A_all; A[k,:]']
        end
    end
    
    return A_all_f, A_all, dA_all
end

contact_mode_constraints (generic function with 1 method)

In [7]:
function contact_force_constraints(λ, contactMode)
    # c > 0
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    ss_active = ss_mode[cs_mode]
    
    i = 1
    ic = 1
    c = zeros(sum(ss_active.==0)*3 + sum(ss_active.!=0))
    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            c[ic:ic+2] = [-λ[i]; -μ*λ[i] - λ[i+1]; -μ*λ[i] + λ[i+1]]
            ic += 3
            i += 2
        else
            c[ic] = -λ[i]
            i += 1
            ic += 1
        end
    end
    return c
end

contact_force_constraints (generic function with 1 method)

In [8]:
function sliding_velocity_constraints(x, contactMode)
    # reture A_eq, A_geq, A_eq*dq = 0, A_geq*dq >= 0
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    AA_eq = zeros(0,3)
    AA_geq = zeros(0,3)
    dAA_eq = zeros(0,3)
    dAA_geq = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            AA_eq = [AA_eq; A[k,:]'; A_t[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]'; dA_t[k,:]']
        else
            AA_eq = [AA_eq; A[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]']
            AA_geq = [AA_geq; ss*A_t[k,:]']
            dAA_geq = [dAA_geq; ss*dA_t[k,:]']
        end
    end
    
    return AA_eq, AA_geq, dAA_eq, dAA_geq
end

sliding_velocity_constraints (generic function with 1 method)

In [9]:
function solveEOM(x, contactMode, controller)
    # contactMode: bool vector, indicates which constraints are active
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    if typeof(controller) == typeof(zeros(3))
        Y = controller
    else
        Y = controller(x, contactMode)
    end
    
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    #z = blockMat\b
    #println(blockMat)
    if rank(blockMat) < length(b)
        z =pinv(blockMat)*b
    else
        z = blockMat\b
    end
    
    ddq = z[1:3]
    if (sum(contactMode[1:n_contacts])>=1)
        λ = z[4:end]
    else
        λ = []
    end
    
    return ddq, λ
end

solveEOM (generic function with 1 method)

In [10]:
function computeResetMap(x, contactMode)
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    if rank(blockMat) < 3+c
        z = pinv(blockMat)*[M*dq; zeros(c)]
    else
        z = blockMat\[M*dq; zeros(c)]
    end
        
    dq_p = z[1:3]
    p_hat = z[4:end]
    return dq_p, p_hat
end

computeResetMap (generic function with 1 method)

In [11]:
function compute_FA(x, controller)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< 1e-6
    inactive_cs = abs.(a) .> 1e-6
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end

    max_cons = 0
    
    for kk = 1:size(possibleModes, 1)      
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, dA_separate = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        ddq, λ = solveEOM(x, m, controller)
        
        c_λ = contact_force_constraints(λ, m)
        
        if all(c_λ.>=0)
        
            As_eq, As_geq, dAs_eq, dAs_geq = sliding_velocity_constraints(x, m)

            sep_vel_cond = ((A_separate*dq).>0) .| ((A_separate*ddq .+ dA_separate*dq).>=0)
            maintain_vel_cond = all(abs.(As_eq*dq).<1e-6) & all((As_geq*dq).>1e-6)

            if ~maintain_vel_cond
                if any((As_geq*dq).<1e-6)
                    maintain_vel_cond = all(abs.(dAs_eq*dq + As_eq*ddq).<1e-6) & all((dAs_geq*dq + As_geq*ddq).>0)
                end
            end

            if all(c_λ.>=0) && all(sep_vel_cond) && maintain_vel_cond
                if sum(m[1:n_contacts]) > max_cons
                    contactMode = m
                    max_cons = sum(m[1:n_contacts])
                end
            end
        end
    end
    
    return contactMode
end

compute_FA (generic function with 1 method)

In [12]:
function compute_IV(x)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< 1e-6
    inactive_cs = abs.(a) .> 1e-6
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end
    
    max_cons = 0
    for kk = 1:size(possibleModes, 1)
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        dq_p, p_hat = computeResetMap(x, m)
        
        c_p_hat = contact_force_constraints(p_hat, m)
        As_eq, As_geq, _, _ = sliding_velocity_constraints(x, m)
        
        if all(c_p_hat.>=0) && all(A_separate*dq_p.>=0) && all(abs.(As_eq*dq_p).<1e-6) && all(As_geq*dq_p.>0)
             if sum(m[1:n_contacts]) > max_cons
                contactMode = m
                max_cons = sum(m[1:n_contacts])
            end
        end
    end
    
    return contactMode
end

compute_IV (generic function with 1 method)

In [13]:
function mode(x, controller)
    return compute_FA(x, controller)
end

mode (generic function with 1 method)

In [14]:
function guard_conditions(x, contactMode, controller)
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    a[contactMode[1:n_contacts] .== 1] .= 0.0
    
    v_all = zeros(n_contacts)
    _, As_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    v_all[1:size(As_geq,1)] = -As_geq*dq
    
    ddq, λ = solveEOM(x, contactMode, controller)
    c_λ = contact_force_constraints(λ, contactMode)
    c_λ_all = zeros(3*n_contacts)
    c_λ_all[1:length(c_λ)] = c_λ
    
    c = [a; v_all; c_λ_all]
    
    dir = [-ones(Int,length(a)); ones(Int,length(v_all)); ones(Int,length(c_λ_all))]
    
    return c, dir
end

guard_conditions (generic function with 1 method)

## Wrappers for ODE solver

In [15]:
function fixed_control(x, contactMode)
    return f_fixed
end

fixed_control (generic function with 1 method)

In [16]:
function dynamics!(dx, x, p, t)
    # p from integrator: (contact mode, controller, t_control, h_control, u_control)
    println("Dynamics evalutation ", t)
    q = x[1:3]
    dq = x[4:6]
    contactMode = p[1]
    controller = p[2]
    t_control = p[3][1]
    h_control = p[4]
    u_control = p[5]
    
    if t > (t_control + h_control)
        println("Control evaluation")
        p[3] .= [Float64(t)]
        u_control = controller(x, contactMode)
        p[5] .= u_control
    end
    
    ddq, λ = solveEOM(x, contactMode, u_control)
    dx .= [dq; ddq]
end

dynamics! (generic function with 1 method)

In [17]:
function conditions(out, x, t, integrator)
    contactMode = integrator.p[1]
    controller = integrator.p[2]
    c, dir = guard_conditions(x, contactMode, controller)
    out .= c
end

function affect!(integrator, idx)
    contactMode = integrator.p[1]
    x = integrator.u
    c, dir = guard_conditions(x, contactMode, integrator.p[2])
    
    # only consider upcrossing forces and constraints values(FA comp)
    # forces
    if dir[idx] > 0
        new_contactMode = compute_FA(x, integrator.p[2])
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        println("New mode from FA: ", new_contactMode)
    end
    # constraints
#     if dir[idx] < 0
#         new_contactMode = contactMode
#         new_contactMode[idx] = false
#     end
end

function affect_neg!(integrator, idx)
    contactMode = integrator.p[1]
    x = integrator.u
    c, dir = guard_conditions(x, contactMode, integrator.p[2])
    
    # only consider down crossing constraint value(IV comp)
    if dir[idx] < 0
        new_contactMode = compute_IV(x)
        dq_p, p_hat = computeResetMap(x, new_contactMode)
        integrator.u .= [x[1:3]; dq_p]
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        println("New mode from IV: ", new_contactMode)

    end
end

affect_neg! (generic function with 1 method)

## goal specific conditions

In [18]:
# reference conditions
xref = [0;h/2;0;1;0;0]
uref = [μ*m*g;0]
nominal_mode = [1 1 1 1]

# pusher location
pusher_p = [-w/2;0]

2-element Vector{Float64}:
 -0.5
  0.0

## Basic controllers

In [19]:
function no_controller(x, contactMode)
    return zeros(3)
end

function force_controller(x, contactMode)
    f = zeros(3)
    f[1] = 1.2*μ*m*g
    return f
end

function pd_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    k = 10
    d = 1
    eq = q - xref[1:3]
    eq[1] = 0
    f = -k.*eq .- d.*(dq - xref[4:6])
    
    # add sliding force compensation
    if abs(f[1]) > 1e-3
        f[1] += μ*m*g
    end
    return f
end

function pd_pusher_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    
    k = 10
    d = 1
    
    A = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    eq = q - xref[1:3]
    eq[1] = 0
    fd = -k.*eq .- d.*(dq - xref[4:6])
    
    # add sliding force compensation
    if abs(fd[1]) > 1e-3
        fd[1] += μ*m*g
    end
    
    c = A\fd
    f = A*c
    return f
end
    

pd_pusher_controller (generic function with 1 method)

## LQR 
Push the box with a pusher (a point contact)

In [20]:
"""
    riccati(A,B,Q,R,Qf,N)

Use backward riccati recursion to solve the finite-horizon time-invariant LQR problem.
Returns vectors of the feedback gains `K` and cost-to-go matrices `P`, where `length(K) == N-1`,
`length(P) == N`, and `size(K[i]) == (m,n)` and `size(P[i]) == (n,n)`.

# Arguments:
* `A`: `(n,n)` discrete dynamics Jacobian wrt the state
* `B`: `(n,m)` discrete dynamics Jacobian wrt the control
* `Q`: `(n,n)` stage-wise cost matrix for states
* `R`: `(m,m)` stage-wise cost matrix for controls
* `Qf`: `(n,n)` cost matrix for terminal state
* `N`: Integer number of time steps (horizon length).
"""
function riccati(A,B,Q,R,Qf,N)
    # initialize the output
    n,m = size(B)
    P = [zeros(n,n) for k = 1:N]
    K = [zeros(m,n) for k = 1:N-1]
    
    # TODO: implement the Riccati recursion
    P[end] .= Qf
    for k = reverse(1:N-1) 
        K[k] .= (R + B'P[k+1]*B)\(B'P[k+1]*A)
        P[k] .= Q + A'P[k+1]*A - A'P[k+1]*B*K[k]
    end
    
    # return the feedback gains and ctg matrices
    return K,P
end

riccati

In [21]:
function rk4_step(f,xk,u)
    # classic rk4 parameters
    a21 = 0.5
    a31 = 0
    a32 = 0.5
    a41 = 0
    a42 = 0
    a43 = 1
    b1 = 1/6
    b2 = 1/3
    b3 = 1/3
    b4 = 1/6
    
    f1 = f(xk,u)
    f2 = f(xk + Δt*a21*f1,u)
    f3 = f(xk + Δt*a31*f1 + Δt*a32*f2,u)
    f4 = f(xk + Δt*a41*f1 + Δt*a42*f2 + Δt*a43*f3,u)

    xn = xk + Δt*(b1*f1 + b2*f2 + b3*f3 + b4*f4)
    
    return xn
end

function continuous_dynamics(x, u, mode)
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, mode)
    
    # contact constraints
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = Jc*u
    
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    if rank(blockMat) < length(b)
        z =pinv(blockMat)*b
    else
        z = blockMat\b
    end

    ddq = z[1:3]
    
    return [dq;ddq]
end

function continuous_dynamics_for_lqr(x, u, mode)
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, mode)
    
    # contact constraints
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = Jc*u
    
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    z = blockMat\b

    ddq = z[1:3]
    
    return [dq;ddq]
end

function discrete_dynamics(x, u, mode)
    xn = rk4_step((_x,_u)->continuous_dynamics_for_lqr(_x,_u,mode),x,u)
    return xn
end

discrete_dynamics (generic function with 1 method)

In [22]:
# compute infinite horizon K
A_lqr = ForwardDiff.jacobian(_x->discrete_dynamics(_x, uref, nominal_mode), xref)
B_lqr = ForwardDiff.jacobian(_u->discrete_dynamics(xref, _u, nominal_mode), uref)
Q_lqr = Diagonal([0;1.0;1.0;1.0;1.0;1.0])
R_lqr = Diagonal([0.1;0.1])
Ks, _ = riccati(A_lqr,B_lqr,Q_lqr,R_lqr,Q_lqr,50)
K_lqr = Ks[1]

2×6 Matrix{Float64}:
 0.0  0.0  0.0  2.58945  0.0  0.0
 0.0  0.0  0.0  1.29473  0.0  0.0

In [23]:
function pusher_lqr_controller(x, contactMode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = uref .- K_lqr*(x .- xref)
    f = Jc*u
    return f
end

pusher_lqr_controller (generic function with 1 method)

## Hybrid controller

### Hybrid system definition

In [24]:
# domain
# ineqs > 0; eqs = 0
function domain(x, contactMode)
    
    cs = contactMode[1:n_contacts] .== 1
    ss = contactMode[n_contacts+1:end]
    
    q = x[1:3]
    dq = x[4:6]
    
    # a(q) = or > 0
    
    a = compute_a(q)
    
    eqs_a = a[cs]
    ineqs_a = a[.~cs]
    
    # A_eq dq = 0; A_geq dq > 0
    
    A_eq, A_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    
    eqs_A = A_eq*dq
    ineqs_A = A_geq*dq
    
    # separating contacts
    separate_cs = (cs.!=1) .& (abs.(a) .< 1e-6)
    _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)])
    
    ineqs_A_sep = A_separate*dq
    
    ineqs = [zeros(0); ineqs_a; ineqs_A]#; ineqs_A_sep]
    eqs = [zeros(0); eqs_a; eqs_A]
    
    return ineqs, eqs
end

domain (generic function with 1 method)

In [25]:
function guard_set(x, mode_from, mode_to)
    #Todo
    
    ineqs = zeros(0)
    eqs = zeros(0)
    
    q = x[1:3]
    dq = x[4:6]
    
    d_ineqs, d_eqs = domain(x, mode_from)
    # c: d_ineqs > 0, d_eqs = 0
    ineqs = [ineqs; d_ineqs]
    eqs = [eqs; d_eqs]
    
    cs_mode_to = mode_to[1:n_contacts].==1
    cs_mode_from = mode_from[1:n_contacts].==1
    
    A_f_new, A_new, dA_new = contact_mode_constraints(x, mode_to)
    
    # if there is new impact
    new_cs = (cs_mode_from.==false).& (cs_mode_to.==true)
    if sum(new_cs) > 0
            a_new = compute_a(x)
        # c: a_new[new_cs] .== 0
        eqs = [eqs; a_new[new_cs]]
        
        _, A_impact, _, = contact_mode_constraints(x, [new_cs; ones(n_contacts)])
        #c: A_impact*dq < 0
        ineqs = [ineqs; -A_impact*dq]
        
        p_hat = pinv(A_new*inv(M)*A_f_new')*A_new*dq
        #c: contact_force_constraints(p_hat, mode_to) > 0
        ineqs = [ineqs; contact_force_constraints(p_hat, mode_to)]
    end
    
    # if it is stick -> slide: do nothing, guard = domain
    # if it is slide -> stick: A_stick dq = 0
    ss_mode_to = mode_to[n_contacts+1:end]
    ss_mode_from = mode_from[n_contacts+1:end]
    ss_new_stick = (cs_mode_from.==true).& (cs_mode_to.==true) .& (ss_mode_to .== 0) .& (ss_mode_from .!=0)
    
    if sum(ss_new_stick) > 0    
        A_stick, _, _, _ = sliding_velocity_constraints(x, [ss_new_stick; zeros(n_contacts)])
        # c: A_stick*dq = 0
        eqs = [eqs; A_stick*dq]
    end
        
    return ineqs, eqs
end

guard_set (generic function with 1 method)

In [26]:
function jumpmap(x, mode_from, mode_to)
    dq_p, p_hat = computeResetMap(x, mode_to)
    return [x[1:3]; dq_p]
end

jumpmap (generic function with 1 method)

In [46]:


function V(x)
    q = x[1:3] + x[4:6]
    dq = x[4:6]
    v = 0.5*([q;dq] .- xref)'*Q_lqr*([q;dq] .- xref)
    return v
end 

function dVdx(x)
    res = ForwardDiff.gradient(V, x)
    return res
end

dVdx (generic function with 1 method)

In [47]:

# z: optimizaiton vars
# x: system state
function hybrid_objective(z, x, uref)
    u = z[1:2]
    β = z[3:end]
    l = (u .- uref)'*(u .- uref) + β'*β
    return l
end

function hybrid_constraints!(c,z,x)
    
    u = z[1:2]
    β = z[3:end]
    α = 0.5
    
    Vx = dVdx(x)
    Vv = V(x)
    
    n_modes = size(modes,1)
    
    β_idx = 1
    
    cc = zeros(eltype([c;z;x]),n_modes*2-1)
  
    # mode dynamics
    for k = 1:n_modes
    
        m = modes[k,:]
        
        d_ineq, d_eq = domain(x, m)
        
        n_ineq = size(d_ineq, 1)
        n_eq = size(d_eq, 1)
        
        c_dynamics = -(Vx'*continuous_dynamics(x, u, m) + α*Vv) 
                    - β[β_idx:β_idx+n_ineq-1]'*d_ineq 
                    - β[β_idx+n_ineq:β_idx+n_ineq+n_eq-1]'*d_eq 
                    - β[β_idx+n_ineq+n_eq:β_idx+n_ineq+2*n_eq-1]'*(-d_eq)
        
        β_idx += n_ineq + 2*n_eq
        
        cc[k] = c_dynamics
    end
    
    nominal_mode = modes[10,:]
    
    for k = 1:(n_modes-1)
        
        mode_from = modes[k,:]
        
        d_ineq, d_eq = guard_set(x, mode_from, nominal_mode)
        
        n_ineq = size(d_ineq, 1)
        n_eq = size(d_eq, 1)

        c_jump = -(Vx'*continuous_dynamics(jumpmap(x, mode_from, nominal_mode), u, nominal_mode) + α*Vv) 
            - β[β_idx:β_idx+n_ineq-1]'*d_ineq 
            - β[β_idx+n_ineq:β_idx+n_ineq+n_eq-1]'*d_eq 
            - β[β_idx+n_ineq+n_eq:β_idx+n_ineq+2*n_eq-1]'*(-d_eq)
        
        β_idx += n_ineq + 2*n_eq
        
        cc[n_modes+k] = c_jump
        
    end
    
    c .= cc
        
#     println("Total β: ", β_idx, "total constraints: ", size(cc)) total β = 173
    
end

nonnegative_constraint_indices = (1:19)

function primal_bounds(n)
    # Enforce simple bound constraints on the decision variables (e.g. positivity) here
    # x_l ≤ [u; β] ≤ x_u
    
    x_l = [-Inf*ones(2);zeros(n-2)]
    x_u = [20*ones(2);Inf*ones(n-2)]

    return x_l, x_u
end

primal_bounds (generic function with 1 method)

### Interface with IPOPT

In [48]:
struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    objective
    constraint!
    primal_bounds
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    constraint_bounds
    hessian_lagrangian::Bool
    params
end

function ProblemMOI(n_nlp,m_nlp,objective, constraint!, primal_bounds;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false,
        params = (zeros(6),zeros(2)))

    ProblemMOI(n_nlp,m_nlp,
        objective, constraint!,
        primal_bounds,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        constraint_bounds,
        hessian_lagrangian,
        params)
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)

    c_u = zeros(m)
    c_u[idx_ineq] .= Inf
    
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    prob.objective(x, prob.params[1], prob.params[2])
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,_x->prob.objective(_x,prob.params[1], prob.params[2]) ,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    prob.constraint!(g, x, prob.params[1])
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    
    cons! = (_c, _z)->prob.constraint!(_c, _z, prob.params[1])
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), cons!, zeros(prob.m_nlp), x)
    
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function ipopt_solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-6,c_tol=1.0e-6,max_iter=1000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol
    
    #Uncomment the following line to turn off verbose IPOPT output
    #solver.options["print_level"] = 0

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # no verbose output
    MOI.set(solver, MOI.Silent(), true)
    
    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

ipopt_solve (generic function with 1 method)

In [49]:
#TODO: global variable. define hybrid_prob
n_nlp = 2+173
m_nlp = 19 # total number of constraints
hybrid_prob = ProblemMOI(n_nlp,m_nlp, hybrid_objective, hybrid_constraints!, primal_bounds(n_nlp), idx_ineq=nonnegative_constraint_indices);

### define the controller

In [50]:
function hybrid_reference_lqr(x, contactMode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = uref .- K_lqr*(x .- xref)
    return u
end

hybrid_reference_lqr (generic function with 1 method)

In [53]:
function hybrid_controller(x, contactMode)
    
    u_ref = hybrid_controller_reference(x, contactMode) # from reference controller
    #println(u_ref)
    hybrid_prob.params[1] .= x
    hybrid_prob.params[2] .= u_ref
    
    z_guess = [u_ref; zeros(hybrid_prob.n_nlp-2)]
    
    z_sol = ipopt_solve(z_guess,hybrid_prob);
    
    u = z_sol[1:2]
    #println(hybrid_objective(z_sol, x, u_ref))
    #println(z_sol)
    #cc = zeros(19)
    #hybrid_constraints!(cc, z_sol, x)
    #println(cc)
    
    # return space wrench
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    f = Jc*u
    
    return f
    
end

hybrid_controller (generic function with 1 method)

In [54]:
println(pusher_lqr_controller(x0, initial_mode))
println(hybrid_controller(x0, initial_mode))

[7.494454027157854, 1.2947270135789266, -0.6473635067894633]
[11.402953057464112, 1.8124203224447728, -0.9062101612223864]


In [ ]:
1.3045/7.5380 - 1.2947/7.4944540

In [ ]:
xref

In [ ]:
x0

## simulate the sliding box 

In [35]:
# specify the controller
controller = hybrid_controller
hybrid_controller_reference = hybrid_reference_lqr


h_control = Δt

0.05

In [38]:
# initial condition

# # basic test
# x0 = [0;h/2;0;0;0;0]

# lqr fail 1
θ0 = -0.5
x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]
initial_mode = mode(x0, controller)
# # lqr fail 2
# θ0 = -0.5
# x0 = [0;h/2 - sin(θ0)*w/2 + 0.2;θ0;0;0;0]

4-element Vector{Int64}:
 0
 0
 0
 0

In [ ]:



tspan = (0.0, 1.0)
callback_length = 5*n_contacts

prob = ODEProblem(dynamics!, x0, tspan, (initial_mode, controller, [0.0], h_control, zeros(3)))
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
# sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15, adaptive=false,dt=Δt)
sol = solve(prob, Tsit5(); callback = cb, adaptive=false,dt=Δt)
println("Simulation status: ", sol.retcode)

In [ ]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

In [ ]:
# animation 
n = length(sol.t)
x = zeros(n)
y = zeros(n)
θ = zeros(n)
for k = 1:n
    x[k] = sol.u[k][1]
    y[k] = sol.u[k][2]
    θ[k] = sol.u[k][3]
end
    
anim = @animate for i ∈ 1:n
    plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-3,3), ylims=(-0.5,3))
    plot!(boxshape([x[i],y[i],θ[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
    
end

In [ ]:
gif(anim, "anim.gif", fps = Int(floor(0.5*n/tspan[2])));

In [ ]:
sol.u